In [ ]:
import pickle
import numpy
from collections import OrderedDict
import zmq
import sys
import threading
import time
from random import randint, random
import zmq
import warnings
warnings.filterwarnings('ignore')
numpy.set_printoptions(suppress=False)
import base64
import torch
import torchvision
import torch.nn as nn
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from torchsummary import summary
import numpy as np
import torchvision.utils as vutils
from torch import nn, optim
from torch.nn import functional as F
import math
torch.set_printoptions(sci_mode=False)

def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])


class FCN(nn.Module):
    def __init__(self):
        super(FCN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
        return x

D = FCN()


################################################################
D.load_state_dict(torch.load('fcn_mnist.ckpt'))
discrimiator = D.state_dict()
    

vals111 = []
keys = []
vals = []
for k, v in discrimiator.items():

    keys.append(k)
    vals.append(v)
    a = v.numpy()
    vals111.append(a.shape)
    

###################################################################
def elapsed_time_total(start, end):
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    print("Total Traning Time: {:0>2}:{:0>2}:{:05.2f}"
                .format(int(hours),int(minutes),seconds))

def elapsed_time_avg(start, end):
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    print("Averaging overhead: {:0>2}:{:0>2}:{:05.2f}"
                .format(int(hours),int(minutes),seconds))

# Assuming UTF-8 encoding, change to something else if you need to
base64.b64encode("password".encode("utf-8"))


def write_data(file_name, data):
    if type(data) == bytes:
        #bytes to base64
        data = base64.b64encode(data)
         
    with open(file_name, 'wb') as f: 
        f.write(data)
 
def read_data(file_name):
    with open(file_name, "rb") as f:
        data = f.read()
    #base64 to bytes
    return base64.b64decode(data)
###################################################################


global data_list
global client_num

client_num = 0

context = zmq.Context()
socket = context.socket(zmq.ROUTER)
socket.bind("tcp://*:5555")



pub_socket = context.socket(zmq.PUB)
pub_socket.bind("tcp://*:5557")


start_total = time.time()

print("The server is running now!")

c = 0
data_list = []
loaded_enc = []
loaded_enc_tmp = []
cipher1 = []
sum_ = 0

data_list_dicriminator = []
sum_1 = 0


#number of clients
client_num_ = 3
#number of global iterations
global_iter = 10


while c < client_num_ * global_iter:
    
    ident1, msg1 = socket.recv_multipart()
        
    string = b"New"
    
    if string == msg1: 
        
        client_num = client_num + 1
        
        message1 = pickle.dumps(vals)
        socket.send_multipart([ident1, message1])
        
        print("client_num: ", client_num)
        
        print("Base model sent to the new client!")
    
    else: 
        
        print("Training round started")
        
        message1 = pickle.loads(msg1)
        
            
        data_list_dicriminator.append(message1)


        if len(data_list_dicriminator) == client_num_:
            
            print("Enough data recevied")
            
            start_avg = time.time()
    
        
            cipher1 = [sum(x) for x in zip(*data_list_dicriminator)]
            cipher1_avg = [value / client_num_ for value in cipher1]
            

 

            print("Avgg generator encrypted computed")
            end_avg = time.time()
            elapsed_time_avg(start_avg, end_avg) 
            
            message1 = pickle.dumps(cipher1_avg)
            print("Plain data size in bytes {}".format(convert_size(len(message1))))
            
            pub_socket.send(message1)
            
            print("Sent!")
            

            cipher1 = []
            sum_1 = 0
            sum_final1 = 0
            data_list_dicriminator = []
        
        print("c: ", c)
        c = c + 1
        
end_total = time.time()
elapsed_time_total(start_total, end_total) 